In [5]:
!pip install langchain_community
!pip install langchain
!pip install pypdf
!pip install chromadb

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 7.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached setuptools-69.5.1-py3-none-any.whl.metadata (6.2 kB)
  Using cached click-8.1.7-py3-none-any.whl.metadata (3.0 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.8/526.8 kB 6.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.5/506.5 kB 13.2 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 10.9 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 9.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.8

In [6]:
import os
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma

OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
DOC_PATH = "../docs/openid_action.pdf"
CHROMA_PATH = "docs" 

# load your pdf doc
loader = PyPDFLoader(DOC_PATH)
pages = loader.load()

# chunk the content
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = text_splitter.split_documents(pages)

embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

# embed the chunks as vectors and load them into the database
db_chroma = Chroma.from_documents(chunks, embeddings, persist_directory=CHROMA_PATH)


In [7]:
query = 'what is the openID Connect?'

# retrieve context - top 5 most relevant (closests) chunks to the query vector 
# (by default Langchain is using cosine distance metric)
docs_chroma = db_chroma.similarity_search_with_score(query, k=5)

# generate an answer based on given user query and retrieved context information
context_text = "\n\n".join([doc.page_content for doc, _score in docs_chroma])

print(context_text)

your life you have used OpenID Connect to log into some web or mobile application.  If you 
have ever used Log in with Apple ID or Log in with Google , you have used OpenID 
Connect underneath. 1 
In simple terms, OpenID Connect is a standard developed by the OpenID foundation that 
defines how a client application communicates with an identity  provider to identify a user.2 A 
client application can be a single -page application, a native mobile application, a server -side

synonymous to OpenID Connect – they are not. OpenID is a specification developed by 
OpenID foundation (the same entity behind OpenID Connect) in 2005, as a way of facilitating login to a client application (OpenID uses the term relying party  to represent a client 
application), via an identity provider the user picks. The e nd user experience is very much 
similar to what you experience today when you login to eBay using your Apple ID or Google

A and B of the book help you get a jumpstart with React and React Na

In [8]:
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI

# you can use a prompt template
PROMPT_TEMPLATE = """
    Answer the question based only on the following context:
    {context}
    Answer the question based on the above context: {question}.
    Provide a detailed answer.
    Don’t justify your answers.
    Don’t give information not mentioned in the CONTEXT INFORMATION.
    Do not say "according to the context" or "mentioned in the context" or similar.
"""

# load retrieved context and user query in the prompt template
prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
prompt = prompt_template.format(context=context_text, question=query)

# call LLM model to generate the answer based on the given context and query
model = ChatOpenAI()
response_text = model.predict(prompt)

print(response_text)


/Users/claudioed/development/ai/langchain-intro/.venv/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(
/Users/claudioed/development/ai/langchain-intro/.venv/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseChatModel.predict` was deprecated in langchain-core 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


OpenID Connect is a standard developed by the OpenID foundation that defines how a client application communicates with an identity provider to identify a user. It is built on top of the OAuth 2.0 specification and serves as an identity layer for authentication. OpenID Connect facilitates single sign-on, identity federation, attribute sharing, single logout, and other use cases. It is not compatible with the older OpenID standard and is widely used in modern applications for secure and efficient user authentication.
